In [1]:
# Konjunktion von Formeln (in SMTLIB-Syntax)
def AND(formeln):
    if formeln == []:
        return "true"
    else:
        return "(and " + " ".join(formeln) + ")"
# Disjunktion von Formeln (in SMTLIB-Syntax)
def OR(formeln):
    if formeln == []:
        return "false"
    else:
        return "(or " + " ".join(formeln) + ")"
# Negation einer Formel (in SMTLIB-Syntax)
def NOT(formel):
    return "(not " + formel + ")"
# Implikation formel1 => formel2 (in SMTLIB-Syntax)
def IMPL(formel1,formel2):
    return "(=> " + formel1 + " " + formel2 + ")"

# eine propositional Formel als Input für SAT (in SMTLIB-Syntax)
def SATInput(variablen, formel):
    return "".join(["(declare-fun " + v + " () Bool)\n" for v in variablen]) + "(assert " + formel + ")\n" + "(check-sat)(get-model)"

# reduziert HamiltonPath für Graph G auf SAT, G gegeben als Listen Kno von Knoten und Kan von Kanten (k,l)
def HS(Kno,Kan):
    # die Anzahl der Knoten
    N = len(Kno)
    # die Liste der Positionen im Pfad
    Positionen = range(N)
    # successors[k] ist die Liste der k', so dass G die Kante (k,l) hat
    succ = {}
    for k in Kno:
        succ[k] = []
    for (k,l) in Kan:
         succ[k].append(l)
            
    def Pos(k,p):
        return "pos_" + str(k) + "_" + str(p)
    
    # die Liste der propositionalen Variablen
    variablen = [Pos(k,p) for k in Kno for p in range(N)]
    
    # Bauen der Teil-Formeln
    
    jedePositionHatEinenKnoten = AND([OR([Pos(k,p) for k in Kno]) for p in Positionen])
    
    # nicht k und l an Position p
    def nichtInGleicherPosition(k,l,p):
        return NOT(AND([Pos(k,p), Pos(l,p)]))
    keinePositionHatZweiKnoten = AND([nichtInGleicherPosition(k,l,p) for p in Positionen for k in Kno for l in Kno if k != l])
    
    jederKnotenIstEinmalImPfad = AND([OR([Pos(k,p) for p in Positionen]) for k in Kno])
    
    # nicht k an Positionen p und q
    def nichtAnZweiPositionen(k,p,q):
        return NOT(AND([Pos(k,p), Pos(k,q)]))
    keinKnotenIstZweimalImPfad = AND([nichtAnZweiPositionen(k,p,q) for k in Kno for p in Positionen for q in Positionen if p != q])
    
    # wenn k an Position p, dann successor von k an Position p+1
    def folgePositionHatNachfolger(k,p):
        return IMPL(Pos(k,p), OR([Pos(l,p+1) for l in succ[k]]))
    alleFolgePositionenHabenNachfolger = AND([folgePositionHatNachfolger(k,p) for k in Kno for p in range(N-1)])
    
    # HS(G): Konjunktion aller Teilformeln
    isHamiltonPfad = AND([jedePositionHatEinenKnoten, keinePositionHatZweiKnoten, jederKnotenIstEinmalImPfad, keinKnotenIstZweimalImPfad, alleFolgePositionenHabenNachfolger])
    
    # Formatierung als SMTLIB-Problem
    return SATInput(variablen, isHamiltonPfad)

HS(["A","B","C"], [("A","B"), ("A","C")])

'(declare-fun pos_A_0 () Bool)\n(declare-fun pos_A_1 () Bool)\n(declare-fun pos_A_2 () Bool)\n(declare-fun pos_B_0 () Bool)\n(declare-fun pos_B_1 () Bool)\n(declare-fun pos_B_2 () Bool)\n(declare-fun pos_C_0 () Bool)\n(declare-fun pos_C_1 () Bool)\n(declare-fun pos_C_2 () Bool)\n(assert (and (and (or pos_A_0 pos_B_0 pos_C_0) (or pos_A_1 pos_B_1 pos_C_1) (or pos_A_2 pos_B_2 pos_C_2)) (and (not (and pos_A_0 pos_B_0)) (not (and pos_A_0 pos_C_0)) (not (and pos_B_0 pos_A_0)) (not (and pos_B_0 pos_C_0)) (not (and pos_C_0 pos_A_0)) (not (and pos_C_0 pos_B_0)) (not (and pos_A_1 pos_B_1)) (not (and pos_A_1 pos_C_1)) (not (and pos_B_1 pos_A_1)) (not (and pos_B_1 pos_C_1)) (not (and pos_C_1 pos_A_1)) (not (and pos_C_1 pos_B_1)) (not (and pos_A_2 pos_B_2)) (not (and pos_A_2 pos_C_2)) (not (and pos_B_2 pos_A_2)) (not (and pos_B_2 pos_C_2)) (not (and pos_C_2 pos_A_2)) (not (and pos_C_2 pos_B_2))) (and (or pos_A_0 pos_A_1 pos_A_2) (or pos_B_0 pos_B_1 pos_B_2) (or pos_C_0 pos_C_1 pos_C_2)) (and (not (